In [1]:
from openai import OpenAI
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.options import Options
from bs4 import BeautifulSoup
import time

# OpenAI API 키 설정

client = OpenAI(
    # This is the default and can be omitted
    api_key='API 키 적기',
)

def setup_driver():
    chrome_driver_path = 'chromedriver/chromedriver.exe'  # ChromeDriver 경로를 설정
    chrome_options = Options()
    chrome_options.add_argument("--no-sandbox")
    chrome_options.add_argument("--disable-dev-shm-usage")
    service = Service(chrome_driver_path)
    driver = webdriver.Chrome(service=service, options=chrome_options)
    return driver

def crawl_dynamic_page(url):
    driver = setup_driver()
    driver.get(url)
    time.sleep(3)
    page_source = driver.page_source
    driver.quit()
    return page_source

def extract_text_from_html(html):
    soup = BeautifulSoup(html, 'html.parser')
    text = soup.get_text(separator=' ', strip=True)
    return text

def extract_news_articles(text):
    prompt = f"""
    The following is a text extracted from a website:

    {text}

    Please extract and list the news articles from the above text. The articles should include titles and brief descriptions if available.
    """

    response =client.chat.completions.create(
        model="gpt-4o",
        messages=[
            {"role": "system", "content": "You are a crawrling assistant."},
            {"role": "user", "content": prompt}
        ]
    )            

    return response.choices[0].message.content

# 예제 URL 설정
url = 'https://www.naver.com'

# 크롤링 실행
page_source = crawl_dynamic_page(url)

# 텍스트 추출
extracted_text = extract_text_from_html(page_source)

# 뉴스 기사 추출
news_articles = extract_news_articles(extracted_text)

print(news_articles)


Here are the news articles extracted from the provided text:

1. **"美, 한국산 HBM·반도체장비 對中 수출통제…일본산 장비는 예외"**
   - Description: The United States is implementing export controls on South Korean HBM and semiconductor equipment to China, but Japanese equipment is exempted.

2. **"11월 소비자물가 1.5% 상승…유가 하락에 석달째 1%대, 채소값↑"**
   - Description: Consumer prices increased by 1.5% in November. Despite a decline in oil prices, the figures remain in the 1% range for the third consecutive month, with vegetable prices on the rise.
